In [1]:
import pandas as pd

## pandasのapplyであそぼ

### sampleデータ準備

In [28]:
df = pd.DataFrame([
    ["男性","神奈川県", '川崎市', 11],
    ["男性","東京都", '渋谷区', 34],
    ["男性","神奈川県", '横浜市', 44],
    ["女性","東京都", '品川区', 22],
    ["女性","東京都", '新宿区', 33],
    ["女性","千葉県", '柏市', 22],
    ["不明","千葉県", '千葉市', 60],
    ["不明","東京都", '千代田区', 54],
    ["不明","千葉県", '船橋市', 30],
], columns=['gender','address1', 'address2', 'age'])
df

,gender,address1,address2,age
0,男性,神奈川県,川崎市,11
1,男性,東京都,渋谷区,34
2,男性,神奈川県,横浜市,44
3,女性,東京都,品川区,22
4,女性,東京都,新宿区,33
5,女性,千葉県,柏市,22
6,不明,千葉県,千葉市,60
7,不明,東京都,千代田区,54
8,不明,千葉県,船橋市,30


### DFからSeriesを抽出してapply
- 関数内では、１個分のスカラー値を扱える。

- 単純なreturnだと、Seriesで結果がreturnされる。

In [47]:
%%time
def sample1(x):
    return x*2
df['age'].apply(sample1)

CPU times: user 505 µs, sys: 133 µs, total: 638 µs
Wall time: 645 µs


0     22
1     68
2     88
3     44
4     66
5     44
6    120
7    108
8     60
Name: age, dtype: int64

- Seriesでreturnすると、DFでreturnされる。名前が付けられるので便利だが、ちょっと処理が遅くなる。

In [48]:
%%time
def sample2(x):
    return pd.Series({"age_x2": x*2})
df['age'].apply(sample2)

CPU times: user 7.46 ms, sys: 0 ns, total: 7.46 ms
Wall time: 7.07 ms


,age_x2
0,22
1,68
2,88
3,44
4,66
5,44
6,120
7,108
8,60


- Seriesでreturnは、複数の値を返すこともできる。

In [70]:
%%time
def sample3(x):
    return pd.Series({"age_x2": x*2, "age_x3": x*3})
df['age'].apply(sample3)

CPU times: user 7.74 ms, sys: 0 ns, total: 7.74 ms
Wall time: 7.32 ms


,age_x2,age_x3
0,22,33
1,68,102
2,88,132
3,44,66
4,66,99
5,44,66
6,120,180
7,108,162
8,60,90


- むろん、lambdaでも書ける。

In [72]:
df['age'].apply(lambda x: pd.Series({"age_x2": x*2, "age_x3": x*3}))

,age_x2,age_x3
0,22,33
1,68,102
2,88,132
3,44,66
4,66,99
5,44,66
6,120,180
7,108,162
8,60,90


### DFに直接apply
- DFにapplyする場合、関数内では、1行(もしくは1列)分のSeriesが扱える。
- 行にするためには、axis=1を引数に指定する必要がある。
  - axis=0だと列を扱えるはずだが、あまり使わないかも？

In [74]:
%%time
def sample4(x):
    return pd.Series({"age_x2": x['age']*2})
df.apply(sample4, axis=1)

CPU times: user 5.63 ms, sys: 0 ns, total: 5.63 ms
Wall time: 5.03 ms


,age_x2
0,22
1,68
2,88
3,44
4,66
5,44
6,120
7,108
8,60


- 行内のすべての値にアクセスできるので、条件分岐もできる。

In [51]:
%%time
def sample5(x):
    if x['gender'] == '女性':
        return pd.Series({"age_mod": "黙秘"})
    else:
        return pd.Series({"age_mod": x['age']})
df.apply(sample5, axis=1)

CPU times: user 7.23 ms, sys: 2.78 ms, total: 10 ms
Wall time: 9.35 ms


,age_mod
0,11
1,34
2,44
3,黙秘
4,黙秘
5,黙秘
6,60
7,54
8,30


- 戻り値を複数返すことももちろんできる。

In [66]:
%%time
def sample6(x):
    if x['gender'] == '女性':
        return pd.Series({"age_mod": "黙秘", "message": "黙秘します。"})
    else:
        return pd.Series({"age_mod": x['age'], "message": f"年齢は{x['age']}歳です。"})
df.apply(sample6, axis=1)

CPU times: user 8.51 ms, sys: 99 µs, total: 8.61 ms
Wall time: 8.39 ms


,age_mod,message
0,11,年齢は11歳です。
1,34,年齢は34歳です。
2,44,年齢は44歳です。
3,黙秘,黙秘します。
4,黙秘,黙秘します。
5,黙秘,黙秘します。
6,60,年齢は60歳です。
7,54,年齢は54歳です。
8,30,年齢は30歳です。


### groupbyにapply
- groupybyにapplyする場合、関数内では、条件に該当する部分のDFが扱える。

In [64]:
def sample7(x):
    return pd.Series({"age_mean": x['age'].mean()})
df.groupby('address1', as_index=False).apply(sample7)

,address1,age_mean
0,千葉県,37.333333
1,東京都,35.750000
2,神奈川県,27.500000


- DFにapplyする時と同様、条件文や複数returnももちろんできる。

In [82]:
def sample8(x):
    male_df = x.query('gender == "男性"')
    female_df = x.query('gender == "女性"')
    unknown_df = x.query('gender == "不明"')
    
    return pd.Series({
        "age_mean_male": male_df['age'].mean(),
        "age_mean_female": female_df['age'].mean(),
        "age_mean_unknown": unknown_df['age'].mean(),
    })
df.groupby('address1', as_index=False).apply(sample8)

,address1,age_mean_male,age_mean_female,age_mean_unknown
0,千葉県,NaN,22.0,45.0
1,東京都,34.0,27.5,54.0
2,神奈川県,27.5,NaN,NaN
